In [ ]:
# required libraries & dataset

!curl -L "https://www.dropbox.com/s/ypvbhkgt4sb40sy/dataset_b.zip?dl=0" > dataset_b.zip; unzip dataset_b.zip; rm dataset_b.zip
!pip install livelossplot

In [ ]:
#Importing necessary libraries

import keras
from keras.models import Model
from keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras import layers, models
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.callbacks
import tensorflow as tf

In [ ]:
#Image Size

IMAGE_SIZE= [224,224]
input_shape = IMAGE_SIZE + [3]
seed = 1432
class_mode = 'binary'
n_classes = 1

In [ ]:
#Setting TPU or CPU/GPU strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
# Define training and validation directories.
TRAIN_DIR = "/content/train"
TEST_DIR = "/content/test"
VALID_DIR = "/content/valid"
batch_size = 16

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        seed=seed,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        VALID_DIR,
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        seed=seed,
        class_mode=class_mode)

classnames = train_generator.class_indices

print(classnames)

In [ ]:
#Image augmentation
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
#Loading pretrained model

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
vggmodel = VGG16(input_shape=input_shape, weights='imagenet', include_top=False)

In [ ]:
vggmodel.input

In [ ]:
print('Number of trainable units:', len(vggmodel.trainable_weights))

In [ ]:
#Freezing pre-trained layers

for layers in (vggmodel.layers)[:19]:
    layers.trainable = False
    
print('Number of trainable units:', len(vggmodel.trainable_weights))

In [ ]:
#Modifying last layer

X= Flatten()(vggmodel.output)
predictions = Dense(1, activation="sigmoid")(X)
VGG16_final = Model(vggmodel.input, predictions)

In [ ]:
#Summary

VGG16_final.summary()

In [ ]:
#Compilation

learning_rate = 0.0001
epochs= 100 

with strategy.scope():
    inputs = layers.Input(shape=(input_shape))
    x = img_augmentation(inputs)
    outputs = VGG16(include_top=True, weights=None, classes=1)(x)

VGG16_final.summary()

VGG16_final.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()]
             )

In [ ]:
#Training

from livelossplot import PlotLossesKeras
from livelossplot.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("VGG16_rev230131.h5",
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=False,
                             save_weights_only=False,
                             mode='auto',
                             save_freq=1)

early = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=10,
                      verbose=1, 
                      restore_best_weights=True,
                      mode='auto')

In [ ]:
%%time

vgg_hist=VGG16_final.fit(train_generator,
                        epochs=epochs,
                        validation_data=validation_generator, 
                        callbacks= [checkpoint, early, plot_loss_1],
                        verbose=1)

In [ ]:
"""
from google.colab import files
files.download('/content/VGG16_rev230131.h5')
"""